# 从老版本的Chain迁移到LCEL
+ 示例1：从LLMChain迁移
+ 示例2：从StuffDocumentsChain迁移

In [1]:
import os
from dotenv import load_dotenv

# 加载 .env 文件中的环境变量
load_dotenv(override=True)  # 使用 override=True 确保加载最新的 .env 数据

True

## 从LLMChain迁移到LCEL

### 废弃的用法

In [ ]:
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=os.environ.get("OPENAPI_MODEL"),
    base_url=os.environ.get("OPENAPI_API_BASE"),
    api_key=os.environ.get("OPENAPI_API_KEY"),
    temperature=0,
)

prompt = ChatPromptTemplate.from_messages(
    [("user", "Tell me a {adjective} joke")],
)

# 过时的预制Chain使用
legacy_chain = LLMChain(llm=llm, prompt=prompt)

legacy_result = legacy_chain({"adjective": "funny"})
print(legacy_result)


/tmp/ipykernel_3616149/2859666389.py:16: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  legacy_chain = LLMChain(llm=llm, prompt=prompt)
/tmp/ipykernel_3616149/2859666389.py:18: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  legacy_result = legacy_chain({"adjective": "funny"})


{'adjective': 'funny', 'text': "Sure! Here's a classic for you:  \n\n**Why don’t skeletons fight each other?**  \n*Because they don’t have the guts!*  \n\n(☠️😂)  \n\nWant another one? Just ask!"}


### 使用LCEL重写

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=os.environ.get("OPENAPI_MODEL"),
    base_url=os.environ.get("OPENAPI_API_BASE"),
    api_key=os.environ.get("OPENAPI_API_KEY"),
    temperature=0,
)

prompt = ChatPromptTemplate.from_messages(
    [("user", "Tell me a {adjective} joke")],
)

chain = prompt | llm | StrOutputParser()

chain.invoke({"adjective": "funny"})

"Sure! Here's a classic for you:  \n\n**Why don’t skeletons fight each other?**  \n*Because they don’t have the guts!*  \n\n😄 Hope that gives you a chuckle! Want another?"

## 从StuffDocumentsChain迁移LCEL

### 废弃的用法

In [4]:
from langchain_core.documents import Document
from langchain.chains import LLMChain, StuffDocumentsChain
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

# 文本切割后的文档
documents = [
    Document(page_content="Apples are red", metadata={"title": "apple_book"}),
    Document(page_content="Blueberries are blue", metadata={"title": "blueberry_book"}),
    Document(page_content="Bananas are yellow", metadata={"title": "banana_book"}),
]

document_prompt = PromptTemplate(
    input_variables=["page_content"], template="{page_content}"
)

document_variable_name = "context"

# 总结的提示词模板
prompt = ChatPromptTemplate.from_template("Summarize this content: {context}")
llm = ChatOpenAI(
    model=os.environ.get("OPENAPI_MODEL"),
    base_url=os.environ.get("OPENAPI_API_BASE"),
    api_key=os.environ.get("OPENAPI_API_KEY"),
    temperature=0,
)
llm_chain = LLMChain(llm=llm, prompt=prompt)
chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name=document_variable_name,
    document_prompt=document_prompt,
)

# 使用链
result = chain.invoke(documents)
print(result["output_text"])


/tmp/ipykernel_3616149/3939593960.py:27: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain/
  chain = StuffDocumentsChain(


Here's a concise summary of the content:  

"Apples are red, blueberries are blue, and bananas are yellow."  

This simply lists the colors associated with each fruit mentioned. Let me know if you'd like any modifications!


### 使用LCEL重写

In [ ]:
from langchain_core.documents import Document
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


# 文本切割后的文档
documents = [
    Document(page_content="Apples are red", metadata={"title": "apple_book"}),
    Document(page_content="Blueberries are blue", metadata={"title": "blueberry_book"}),
    Document(page_content="Bananas are yellow", metadata={"title": "banana_book"}),
]

llm = ChatOpenAI(
    model=os.environ.get("OPENAPI_MODEL"),
    base_url=os.environ.get("OPENAPI_API_BASE"),
    api_key=os.environ.get("OPENAPI_API_KEY"),
    temperature=0,
)
# 总结的提示词模板
prompt = ChatPromptTemplate.from_template("Summarize this content: {context}")
chain = create_stuff_documents_chain(llm=llm, prompt=prompt)

chain.invoke({"context": documents})

'Here\'s a concise summary of the given content:  \n\n**"Apples are red, blueberries are blue, and bananas are yellow."**  \n\nThis summarizes the color descriptions of the three fruits in a single sentence. Let me know if you\'d like any modifications!'